# Laboratorium 1 - Algorytmy wyszukiwania wzorców

## Algorytm naiwny

In [1]:
def naive(text, pattern):
    result = []
    for s in range(0, len(text)-len(pattern)+1):
        if (pattern == text[s:s+len(pattern)]):
            result.append(s)
    return result

## Automat skończony

In [2]:
import re
def finite_automation_table(pattern):
    alphabet = set()
    for p in pattern:
        alphabet.add(p)
    
    l_p = len(pattern)
    table = [{} for _ in range(l_p+1)]
    for i in range(l_p+1):
        for a in alphabet:
            k = min(l_p + 1, i + 2)
            while True:
                k = k - 1
                if(re.search(f"{pattern[:k]}$", pattern[:i] + a)):
                    break
            table[i][a] = k
                
    return table
        

In [3]:
def fa_string_matching(text, pattern):
    delta = finite_automation_table(pattern)
    result =[]
    q = 0
    for s in range(0, len(text)):
        q = delta[q].get(text[s])
        if not q:
            q = 0
        elif(q == len(delta) - 1):
            result.append(s+1-q)   
    return result

In [4]:
def fa_string_matching_with_delta(text, delta):
    result =[]
    q = 0
    for s in range(0, len(text)):
        q = delta[q].get(text[s])
        if not q:
            q = 0
        elif(q == len(delta) - 1):
            result.append(s+1-q)
            
    return result

## Algorytm Knutha-Morrisa-Pratta

In [5]:
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        pi.append(k)
    return pi

In [6]:
def kmp_string_matching(text, pattern):
    pi = prefix_function(pattern)
    result = []
    q = 0
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            result.append(i+1-q)
            q = pi[q-1]
            
    return result

## Testy

In [7]:
import time
def measure_time(text, pattern, func):
    start = time.time()
    func(text,pattern)
    end = time.time()
    print(end - start)

### Wyszukiwanie wzorca "Art"

In [8]:
f = open('1997_714.txt', 'r')
text = f.read()
pattern = "Art"

print("Naiwny: ", end="")
measure_time(text, pattern, naive)

print("Automat: ", end="")
measure_time(text, pattern, fa_string_matching)

print("KMP: ", end="")
measure_time(text, pattern, kmp_string_matching)

Naiwny: 0.04386281967163086
Automat: 0.023877859115600586
KMP: 0.02981877326965332


### Naiwny vs KMP

In [9]:
text = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"*10000+"b"
pattern = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"*1000 + "b"

print("Naiwny: ", end="")
measure_time(text, pattern, naive)

print("KMP: ", end="")
measure_time(text, pattern, kmp_string_matching)

Naiwny: 0.8636431694030762
KMP: 0.12372112274169922


### Naiwny vs FA

In [13]:
text = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"*100000+"b"
pattern = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"*20+"b"

print("Naiwny: ", end="")
measure_time(text, pattern, naive)

# żeby czas wyszukiwania był co najmniej 5x dłuższy dla naiwnego wzorzec 
# musi być jeszcze dłuższy, w tym momencie już i tak za długo 
# liczy tablicę przejścia

print("Automat: ", end="")
t = finite_automation_table(pattern)
start = time.time()
fa_string_matching_with_delta(text, t)
stop = time.time()
print(stop-start)

Naiwny: 1.1399545669555664
Automat: 0.8530125617980957


### Obliczenie tablicy przejścia automatu skończonego vs obliczenie funkcji przejścia KMP

In [11]:
pattern = "qwertyuiopasdfghjklzxcvbnm"*20
start = time.time()
finite_automation_table(pattern)
end = time.time()
print("Tablica przejścia: ", end-start)

start = time.time()
prefix_function(pattern)
end = time.time()
print("Funkcja przejścia: ", end-start)

Tablica przejścia:  4.8689916133880615
Funkcja przejścia:  0.0001800060272216797
